In [7]:
from gensim.models import Word2Vec
import pandas as pd
import pickle
import time
import logging
import multiprocessing as mp
logging.basicConfig(
	format='%(asctime)s : %(levelname)s : %(message)s',
	level=logging.INFO)
corpus_path = 'corpus/'
cores = mp.cpu_count()
name_corpus = ['attraction_tag.list',
               'hotel_tag.list',
               'restaurant_tag.list']

name_model = ['model/attraction_tag.model',
              'model/hotel_tag.model',
              'model/restaurant_tag.model']

params_tag = [{'size':300, 'window':99999, 'min_count':0,        # Attraction
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-5},
              {'size':300, 'window':99999, 'min_count':0,        # Hotel
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-5},
              {'size':300, 'window':99999, 'min_count':0,        # Restaurant
               'workers':cores, 'iter':100, 'sg':1, 'sample':1e-5}]

In [9]:
# int to string
for c in name_corpus:
    with open(corpus_path+'kor_'+c, 'rb') as f:
        corpus = pickle.load(f)
    corpus = [[str(pid) for pid in line] for line in corpus]
    with open(corpus_path+'kor_'+c,'wb') as f:
        pickle.dump(corpus, f)
    with open(corpus_path+'eng_'+c, 'rb') as f:
        corpus = pickle.load(f)
    corpus = [[str(pid) for pid in line] for line in corpus]
    with open(corpus_path+'eng_'+c,'wb') as f:
        pickle.dump(corpus, f)

In [11]:
# 한글&영문 corpus 병합
corpora = []
for c in name_corpus:
    with open(corpus_path+'kor_'+c, 'rb') as f:
        kor = pickle.load(f)
    with open(corpus_path+'eng_'+c, 'rb') as f:
        eng = pickle.load(f)
    merged = kor+eng
    with open(corpus_path+c, 'wb') as f:
        pickle.dump(merged, f)
    corpora.append(merged)

In [16]:
#test
corpus = corpora[0]
print('corpus 길이', len(corpus))
total = pd.DataFrame([len(sent) for sent in corpus]).sum()
length = 0
for line in corpus:
    length = max(length, len(line))
print('corpus내 최대 길이', length)
print('corpus내 모든 장소 합', total)

corpus 길이 65862
corpus내 최대 길이 33995
corpus내 모든 장소 합 0    2131509
dtype: int64


In [21]:
spent = []
for i in range(len(corpora)):
    start = time.time()
    model = Word2Vec(corpora[i], **params_tag[i])
    spent.append('Elapsed time: '+str(time.time() - start)+' sec'+' ['+list_tag_model[i]+']')
    model.wv.save(name_model[i])
print(spent)

2018-08-30 18:04:52,928 : INFO : collecting all words and their counts
2018-08-30 18:04:52,929 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-30 18:04:52,954 : INFO : PROGRESS: at sentence #10000, processed 207727 words, keeping 459 word types
2018-08-30 18:04:53,108 : INFO : PROGRESS: at sentence #20000, processed 1970802 words, keeping 527 word types
2018-08-30 18:04:53,127 : INFO : PROGRESS: at sentence #30000, processed 2147057 words, keeping 528 word types
2018-08-30 18:04:53,135 : INFO : PROGRESS: at sentence #40000, processed 2206344 words, keeping 529 word types
2018-08-30 18:04:53,140 : INFO : PROGRESS: at sentence #50000, processed 2233456 words, keeping 529 word types
2018-08-30 18:04:53,145 : INFO : PROGRESS: at sentence #60000, processed 2249499 words, keeping 529 word types
2018-08-30 18:04:53,148 : INFO : collected 531 word types from a corpus of 2256328 raw words and 65862 sentences
2018-08-30 18:04:53,149 : INFO : Loading a fresh voc

2018-08-30 18:04:56,275 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:04:56,276 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:04:56,276 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:04:56,277 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:04:56,277 : INFO : EPOCH - 8 : training on 2256328 raw words (25168 effective words) took 0.4s, 64705 effective words/s
2018-08-30 18:04:56,662 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:04:56,668 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:04:56,670 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:04:56,670 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:04:56,671 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:04:56,671 : INFO : worker thread 

2018-08-30 18:05:00,220 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:05:00,224 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:05:00,230 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:05:00,230 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:05:00,231 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:05:00,231 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:05:00,232 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:05:00,233 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:05:00,233 : INFO : EPOCH - 18 : training on 2256328 raw words (25216 effective words) took 0.4s, 60514 effective words/s
2018-08-30 18:05:00,678 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:05:00,693 : INFO : worker thread

2018-08-30 18:05:03,816 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:05:03,816 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:05:03,817 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:05:03,817 : INFO : EPOCH - 27 : training on 2256328 raw words (25439 effective words) took 0.4s, 66513 effective words/s
2018-08-30 18:05:04,195 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:05:04,200 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:05:04,205 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:05:04,206 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:05:04,207 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:05:04,208 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:05:04,208 : INFO : worker thread

2018-08-30 18:05:07,678 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:05:07,681 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:05:07,685 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:05:07,685 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:05:07,686 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:05:07,686 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:05:07,687 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:05:07,687 : INFO : EPOCH - 37 : training on 2256328 raw words (25307 effective words) took 0.4s, 67497 effective words/s
2018-08-30 18:05:08,064 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:05:08,066 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:05:08,069 : INFO : worker thread

...

2018-08-30 18:06:13,944 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:06:13,944 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:06:13,945 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:06:13,945 : INFO : EPOCH - 68 : training on 1383273 raw words (38037 effective words) took 0.6s, 60420 effective words/s
2018-08-30 18:06:14,558 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:06:14,563 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:06:14,566 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:06:14,568 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:06:14,569 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:06:14,569 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:06:14,570 : INFO : worker thread

2018-08-30 18:06:20,258 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:06:20,264 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:06:20,266 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:06:20,266 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:06:20,267 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:06:20,267 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:06:20,267 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:06:20,268 : INFO : EPOCH - 78 : training on 1383273 raw words (37980 effective words) took 0.6s, 60945 effective words/s
2018-08-30 18:06:20,897 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:06:20,897 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:06:20,900 : INFO : worker thread

2018-08-30 18:06:25,986 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:06:25,986 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:06:25,987 : INFO : EPOCH - 87 : training on 1383273 raw words (38051 effective words) took 0.6s, 59308 effective words/s
2018-08-30 18:06:26,668 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:06:26,671 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:06:26,677 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:06:26,677 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:06:26,678 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:06:26,678 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:06:26,679 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:06:26,679 : INFO : worker thread

2018-08-30 18:06:32,644 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:06:32,645 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-08-30 18:06:32,645 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-08-30 18:06:32,646 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-08-30 18:06:32,646 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-08-30 18:06:32,646 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-08-30 18:06:32,647 : INFO : EPOCH - 97 : training on 1383273 raw words (38314 effective words) took 0.7s, 58457 effective words/s
2018-08-30 18:06:33,296 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-08-30 18:06:33,299 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-08-30 18:06:33,303 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-08-30 18:06:33,304 : INFO : worker thread

KeyboardInterrupt: 